<a href="https://colab.research.google.com/github/anubhavvashishtha/ELL-Assignement/blob/main/examples/llm_inference/bundling/llm_bundling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title License information { display-mode: "form" }
#@markdown Copyright 2024 The MediaPipe Authors.
#@markdown Licensed under the Apache License, Version 2.0 (the "License");
#@markdown
#@markdown you may not use this file except in compliance with the License.
#@markdown You may obtain a copy of the License at
#@markdown
#@markdown https://www.apache.org/licenses/LICENSE-2.0
#@markdown
#@markdown Unless required by applicable law or agreed to in writing, software
#@markdown distributed under the License is distributed on an "AS IS" BASIS,
#@markdown WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#@markdown See the License for the specific language governing permissions and
#@markdown limitations under the License.

In [2]:
#@title Setup { display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
install_out = widgets.Output()
display(install_out)
with install_out:
  !pip install mediapipe
  from mediapipe.tasks.python.genai import bundler

install_out.clear_output()
with install_out:
  print("Setup done.")

Output()

In [1]:
from mediapipe.tasks.python.genai import bundler

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


# Why do we need task bundles?

Executing a text generation pipeline in its entirety requires more than merely employing the core transformer model. It requires preparing the input text to align with the model's required format, running the model autoregressively, and sampling during each iteration. Consequently, once the user has converted their model into `tflite` format (comprising the model's graph and parameters), it becomes essential to augment it with additional metadata to ensure successful end-to-end execution of the model.

Task bundler offers a practical approach to creating such bundles. The example below illustrates how a task bundle can be generated for a converted Gemma model.

In [2]:
tflite_model="/content/gemma3-1b_q4_block128_ekv2048.tflite" # @param {type:"string"}
tokenizer_model="/content/tokenizer.model" # @param {type:"string"}
start_token="<bos>" # @param {type:"string"}
stop_token="<end_of_turn>" # @param {type:"string"}
output_filename="/content/gemma.task" # @param {type:"string"}
enable_bytes_to_unicode_mapping=False # @param ["False", "True"] {type:"raw"}

config = bundler.BundleConfig(
    tflite_model=tflite_model,
    tokenizer_model=tokenizer_model,
    start_token=start_token,
    stop_tokens=[stop_token],
    output_filename=output_filename,
    enable_bytes_to_unicode_mapping=enable_bytes_to_unicode_mapping,
)
bundler.create_bundle(config)

**Notes:**

* The current task pipeline only supports SentencePiece tokenizer models.
* Certain models (e.g., phi-2) use bytes to unicode mapping. Use `enable_bytes_to_unicode_mapping` flag accordingly. Such information, including `start_token` and `stop_tokens` are often provided along with model artifacts.
* The generated output bundle file must end with `.task`. If not, `create_bundle` automatically adds the extension. Do not remove or change that.

In [ ]:
#@title Download the Bundle { display-mode: "form" }
#@markdown Run this cell to download the generated `.task` file.
files.download(output_filename)